# GPU内存

In [1]:
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()

# 你可以添加代码来查看释放后的显存状态
if torch.cuda.is_available():
    print(f"当前已分配显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"当前缓存中显存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("No GPU ")

No GPU 


# 1 准备

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

In [10]:
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# 设置LLM模型
llm = Ollama(model="qwen3:8b", 
            context_window= 8192, #32768,
            # 控制生成文本的最大长度
            max_new_tokens=1024,   #512,
            request_timeout=360.0)
Settings.llm=llm

In [3]:
QA_PROMPT_TEMPLATE_STR = (
    "You are a helpful assistant. Please answer the user's question based on the provided context.\n\n"
    "Context: {context_str}\n"
    "---------------------\n"
    "Question: {query_str}\n"
    "Answer: "
)

In [4]:
from llama_index.core import VectorStoreIndex, PromptTemplate
qa_prompt_template = PromptTemplate(QA_PROMPT_TEMPLATE_STR)

# 2  chat

## 2.1 complete 模式



In [5]:
# 这种方法只适用于单次、无历史的提示
prompt = "写一个关于中国长城的小故事，不超过100字。"

# 使用 .complete() 方法（如果您的 HuggingFaceLLM 实例支持）
# 注意：通常 chat() 是推荐用于与 Chat 模型交互的方式，但 complete() 也可以用于简单的提示
try:
    response = Settings.llm.complete(prompt)
    print("--- LLM Response (Complete) ---")
    print(response.text)
    print("-------------------------------\n")
except AttributeError:
    print("Settings.llm 实例不支持 .complete() 方法，请使用包装后的 .chat()。")


--- LLM Response (Complete) ---
<think>
好的，用户让我写一个关于中国长城的小故事，不超过100字。首先，我需要确定故事的主题和核心元素。长城是中国的象征，所以故事应该突出其历史和文化意义。可能涉及历史人物或者传说，比如孟姜女或者古代士兵。

用户可能希望故事有情感共鸣，所以考虑加入人物情感。比如，一个士兵在长城上的思念，或者建造过程中的艰辛。但字数限制严格，必须简洁。

需要确定故事的结构：开头、发展、结尾。比如，一个守卫在夜晚眺望，回忆过去，最后以象征性的动作结束，比如放飞纸鸢，象征希望或思念。

还要注意字数，确保不超过100字。可能需要精简语言，用生动的意象，比如“月光如银沙”、“烽火台”等。检查是否有冗余，确保每个词都贡献到故事氛围。

最后，确保故事有情感冲击，让读者感受到长城的厚重和人物的情感。可能用“血肉筑起的脊梁”来隐喻长城的建造，同时点出历史的沧桑。
</think>

月光如银沙洒在烽火台，守卫李石望着北方疆界。他记得七年前在此筑墙时，手指被夯土磨出血泡，却始终没让砖缝漏过一粒沙。此刻风掠过城垛，带起他衣襟，仿佛当年那些未及说出口的乡愁，终化作砖石间永恒的纹路。
-------------------------------



## 2.2 chat 模式

### 2.2.1 一次

In [6]:
# 导入 ChatMessage，用于构造对话历史
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings # 确保 Settings 在当前作用域可用

# 假设您想问一个关于中国的简单问题
prompt_content = "写一个关于中国长城的小故事，不超过100字。"

print(f"User Prompt: {prompt_content}\n")

# **关键修改：将字符串包装成 ChatMessage 列表**
messages = [
    ChatMessage(role=MessageRole.USER, content=prompt_content)
]

# 直接调用 llm 实例的 chat 方法，传入 messages 列表
response = Settings.llm.chat(messages)

print("--- LLM Response (Chat) ---")
print(response.message.content)
print("---------------------------\n")

User Prompt: 写一个关于中国长城的小故事，不超过100字。

--- LLM Response (Chat) ---
<think>
好的，用户让我写一个关于中国长城的小故事，不超过100字。首先，我需要确定故事的主题和核心元素。长城是一个历史悠久的建筑，可以结合历史、传说或者人物故事。

用户可能希望故事有情感和画面感，所以得选一个能引起共鸣的情节。比如，可以围绕守卫者、工匠或者历史事件展开。考虑到字数限制，不能太复杂，得简洁有力。

或许用一个守卫者的视角，比如在夜晚巡逻，遇到某种象征性的事件，比如流星，然后引出传说或者历史。这样既有画面感，又能传达长城的精神。

还要注意字数，必须控制在100字以内，所以每个句子都要精炼。可能需要检查是否有冗余的描述，确保每个词都贡献到故事的发展。

另外，用户可能希望故事有教育意义或文化内涵，比如强调坚韧、团结或者历史的厚重。所以结尾部分可以点出长城的象征意义，比如“永恒的脊梁”。

最后，检查语法和流畅度，确保故事紧凑且有感染力。可能需要调整句子结构，让故事在有限的字数内传达深刻的内容。
</think>

夜风掠过箭楼，老卒将最后一块青砖垒上城墙。流星划过天际，他忽然听见砖缝间传来细微的嗡鸣——那是百年前工匠们埋下的蜂巢，此刻正与月光共振，化作一条发光的龙，盘踞在蜿蜒的城墙上。
---------------------------



### 2.2.2 多次

In [7]:
from llama_index.core.chat_engine import SimpleChatEngine

# 使用 Settings 中配置的 llm 创建 ChatEngine
chat_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    # System Prompt 可选，用于设定模型角色
    system_prompt="你是一个乐于助人、充满好奇心的 AI 助手，你的回答总是简洁明了。",
)

print("--- Chat Engine Conversation ---")

# 第一次提问
response_1 = chat_engine.chat("给我推荐一个适合在秋天阅读的中文小说名字。")
print(f"Assistant: {response_1.response}\n")

# 第二次提问（ChatEngine 会自动维护历史）
response_2 = chat_engine.chat("这个作者还有其他的作品吗？")
print(f"Assistant: {response_2.response}\n")

print("--------------------------------\n")

--- Chat Engine Conversation ---
Assistant: <think>
好的，用户让我推荐一个适合秋天阅读的中文小说名字。首先，我需要理解用户的需求。秋天通常给人感觉是宁静、略带忧郁，或者有丰收的喜悦，也可能有离别的氛围。所以推荐的小说应该符合这些情感基调。

接下来，我得考虑哪些中文小说适合这个季节。可能需要找一些描写自然景色、情感细腻或者带有季节元素的故事。比如，余华的《活着》虽然不是秋天的故事，但它的氛围可能和秋天的沉静相呼应。不过用户可能更想要直接关联秋天的作品。

然后，我想到张爱玲的小说，比如《金锁记》，里面有很多关于家庭、命运的描写，可能和秋天的萧瑟感相配。还有汪曾祺的作品，他擅长描绘生活中的小细节，可能适合秋天的氛围。

另外，用户可能希望推荐的是经典作品，或者是比较有文学性的。所以需要确保推荐的小说有较高的文学价值，或者有独特的主题。比如《长恨歌》中的上海，秋天的氛围很浓厚，适合阅读。

还要考虑用户可能的喜好，比如是否喜欢现代小说还是古典文学。如果用户没有明确说明，可能需要推荐一个比较平衡的选择，既有文学性，又容易引起共鸣。比如《城南旧事》中的秋天场景，或者《红楼梦》中的一些章节，虽然不是专门描写秋天，但整体氛围很适合。

另外，用户可能希望小说有情感深度，或者有独特的视角。比如《小团圆》中的情感描写，或者《围城》中的生活百态，可能在秋天阅读时更能体会其中的意味。

最后，确保推荐的小说名字准确，没有错误，并且简要说明推荐理由，让用户明白为什么适合秋天阅读。比如提到秋天的氛围、情感基调，或者小说中的季节元素。这样用户就能理解推荐的原因，而不仅仅是名字。
</think>

《长恨歌》——王安忆  
推荐理由：小说以上海为背景，描绘了主人公在时代变迁中的情感纠葛。秋天的萧瑟与城市的沉静交织，恰如书中人物在岁月长河中的徘徊与坚守，读来令人感怀。

Assistant: <think>
好的，用户之前让我推荐了一本适合秋天阅读的中文小说，我推荐了王安忆的《长恨歌》。现在用户问这个作者还有其他作品吗？我需要列出王安忆的其他作品，并简要说明每本书的特点，以便用户选择。

首先，我得回忆一下王安忆的主要作品。她有很多小说，比如《小城之恋》、《荒山野店》、《女人的勇气》、《天香》、《金锁记》等。每本书的风格和主题可能不同，需要分

## 3 工具调用

In [11]:
llm.metadata

LLMMetadata(context_window=8192, num_output=256, is_chat_model=True, is_function_calling_model=False, model_name='qwen3:8b', system_role=<MessageRole.SYSTEM: 'system'>)

In [13]:
from llama_index.core.memory import ChatMemoryBuffer 
# 确保已安装 llama-index-core

# --- 1. 创建 Memory 对象 ---
# memory 是一个存储对话历史的缓冲区
# token_limit=3900 是一个示例值，应小于您的LLM上下文窗口 (例如 Qwen 7B 是 8192)
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

In [15]:
import asyncio
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings # 您的 LLM 在 Settings.llm 中
# ... (您的 LLM 和 Settings.llm 配置已完成) ...

# ----------------- 定义工具 (假设已成功) -----------------
def get_current_weather(city: str) -> str:
    """获取指定城市（例如：'北京'，'上海'）的当前天气信息。"""
    if "北京" in city:
        return "北京今天的气温是 28°C，多云转晴，空气质量良好。"
    elif "上海" in city:
        return "上海今天的气温是 -18°C，下雨，空气质量一般。"
    return f"抱歉，没有 {city} 的天气数据。"

def multiply(a:int, b:int ) -> int :
    """将两个数相乘(乘法运算)"""
    return a * b + 10000

weather_tool = FunctionTool.from_defaults(fn=get_current_weather)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
tools = [weather_tool, multiply_tool]


agent = ReActAgent(tools=tools, llm=Settings.llm, verbose=True,memory=memory)


In [19]:

# ----------------- 定义同步执行函数 -----------------
def run_agent_queries(agent_instance: ReActAgent):
    print("--- 同步调用 1: 工具函数 (乘法) ---")
    agent_query_calc = "调用乘法工具，计算一下 50 乘以 91 的结果是多少？"
    # 使用 .arun() 方法进行异步执行
    response_calc = agent_instance.query(agent_query_calc)
    print(f"最终回答: {response_calc.response}\n")

    print("--- 同步调用 2: 工具函数 (天气) ---")
    agent_query_weather = "告诉我上海今天的天气怎么样？"
    response_weather = agent_instance.query(agent_query_weather)
    print(f"最终回答: {response_weather.response}\n")
    
run_agent_queries(agent)

--- 同步调用 1: 工具函数 (乘法) ---
> Running step 9a41138a-4eaf-4c60-9583-bba9c8586511. Step input: 调用乘法工具，计算一下 50 乘以 91 的结果是多少？
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 50, 'b': 91}
Observation: 14550
> Running step aa9ab202-01a1-4dd6-9b29-80ddb5d974c5. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 50乘以91的结果是14550。
最终回答: 50乘以91的结果是14550。

--- 同步调用 2: 工具函数 (天气) ---
> Running step 7de30cb7-9471-4b61-80e2-753964284d14. Step input: 告诉我上海今天的天气怎么样？
Thought: The current language of the user is: 中文. I need to use a tool to help me answer the question.
Action: get_current_weather
Action Input: {'city': '上海'}
Observation: 上海今天的气温是 -18°C，下雨，空气质量一般。
> Running step 13b3ffee-ed3f-412b-9095-91766ff0df33. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 上海今天气温为-18°C，有雨，空气质

# Rag1

In [28]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("./txt").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)
response = query_engine.query(  "邓国标工作单位是什么，复旦教育经历的是什么")
print(response)

依据提供的信息，邓国标的当前工作单位是XTransfer（上海夺畅网络技术有限公司），具体职位是CEO兼联合创始人。关于他的复旦教育经历，邓国标曾参加过1998年的上海市高级管理干部培训班（百人工程）第五期，这是一个政府培训班项目，而他并未在复旦大学攻读研究生学位，因此没有列出相应的复旦EMBA或其他硕士项目的经历。他的学历背景为学士学位，是在复旦大学获得的管理科学学士学位。 邓国标参与的复旦教育经历包括：

- 项目/班级/学历/专业：政府培训班/1998上海市高级管理干部培训班(百人工程)第五期/高端培训
- 学历/学位：复旦大学/管理科学/学士学位 To answer the question:

- 邓国标的当前工作单位是XTransfer（上海夺畅网络技术有限公司）。
- 复旦教育经历是：参加了1998年的上海市高级管理干部培训班（百人工程）第五期，这是一个政府培训班项目。
- 他的学历背景为复旦大学管理科学学士学位。 他没有在复旦大学攻读研究生学位，因此没有列出相应的复旦EMBA或其他硕士项目的经历。 邓国标参与的复旦教育经历包括：
  - 项目/班级/学历/专业：政府培训班/1998上海市高级管理干部培训班(百人工程)第五期/高端培训
  - 学历/学位：复旦大学/管理科学/学士学位。


In [21]:
nodes=index.as_query_engine(llm=Settings.llm,text_qa_template=qa_prompt_template).query("用中文回答，施小琳教育经历")

In [22]:
print(nodes)

以下是施小琳的教育经历：
- 复旦管院政府培训班
- 上海市高级管理干部培训班(百人工程)第五期
- 高端培训
这些信息来自提供的文档内容。请注意，这些教育经历的具体细节并未详细列出，仅提及了她参与的一些培训项目。 施小琳在1998年参加了上海市高级管理干部培训班的第五期，并且还参加了复旦大学管理学院的政府培训班以及高端培训项目。这些培训经历显示了她在职业生涯早期就注重个人能力和领导力的提升。 更多具体的信息可能需要进一步查询相关资料。 您如果需要更详细的背景信息或其他方面的数据，请告知我。 根据提供的文档，施小琳的教育经历包括参加复旦大学管理学院的政府培训班和上海市高级管理干部培训班的第五期，还有参加高端培训。 这些培训项目显示了她在职业发展初期就注重提升自己的管理能力和领导力。 具体课程内容未详述。 如果您需要更多详细信息，建议查阅相关的培训记录或档案。 根据文档内容，施小琳的教育经历包括：复旦大学管理学院的政府培训班、上海市高级管理干部培训班（百人工程）第五期以及高端培训。 这些经历表明她在职业生涯早期就积极参与各类管理培训，以提升自身的领导力和管理能力。 更具体的课程详情未提供。 如需更多信息，建议进一步查找相关培训资料。 根据提供的信息，施小琳的教育经历包括参加复旦大学管理学院的政府培训班、上海市高级管理干部培训班（百人工程）第五期和高端培训。 这些培训项目体现了她在管理能力培养方面所做的努力。 更详细的课程内容未在文档中提及。 如果需要更多具体信息，建议查阅相关培训记录。 根据文档内容，施小琳的教育经历如下：她曾参加复旦大学管理学院的政府培训班、上海市高级管理干部培训班（百人工程）第五期和高端培训。 这些经历展示了她在管理能力提升方面的投入。 更详细的课程信息未在文档中给出。 若要获取更多具体信息，建议进一步查找相关培训资料。 以上就是根据文档提供的信息整理出的施小琳的教育经历。 根据提供的文档内容，施小琳的教育经历包括：
- 参加复旦大学管理学院的政府培训班
- 参加上海市高级管理干部培训班（百人工程）第五期
- 参加高端培训
这些培训项目反映了她在管理能力和领导力提升方面的努力。 更具体的课程内容未在文档中提及。 若需更多信息，建议查阅相关培训记录。 以上是根据文档提供的信息整理出的施小琳的教育经历。 根据文档内容，施小琳的教育经历包括：
- 复旦大

In [23]:
len(nodes.source_nodes)

2

In [24]:
nodes.source_nodes[0].text

'以下内容与施小琳有关\n姓名：施小琳\n性别：女\n婚姻状况\t其他\n居住地\t上海市\n更新时间\t2023/03/01\n施小琳标签\t两会代表, 院长级, 二十大人大代表, 第十四届全国人大代表（2023）\n施小琳项目\t政府培训班\n\n施小琳教育经历：复旦管院政府培训班/1998上海市高级管理干部培训班(百人工程)第五期/高端培训\t\n\n默认邮箱\tshi_xiaolin@hotmail.com\n\n施小琳当前工作(工作单位/部门/职位)：四川省省委/副书记;四川省政府/党组书记;四川省人民政府/省长\n\n行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n二类行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n---\n\n施小琳\n工作履历(工作单位/部门/职位)\t上海市普陀区人民政府/区委书记、区委委员;上海市民政局/局长;南汇区人民政府/副区长;虹口区人民政府/副区长;上海市委常委/统战部部长;江西省委/常委;江西省宣传部/部长;四川省成都市/成都市委书记、兼任成都警备区党委第一书记;四川省人民政府/副省长、代理省长\n\n政府类职级\t省部级副职;省部级副职;省部级副职;省部级副职;省部级正职\n\n人际关系(是否校友/姓名/关系/公司/职位/手机号/邮箱/项目班级)\t非校友/陈叶丹/无/无/无/13818353646/无/无\n\n# 施小琳职务变更\n## \t2024/07/31 施小琳任四川省 省长\t成都发布\t\n\n## \t2024/07/04 施小琳任四川省副省长、代理省长\t\t澎湃新闻\t\n\n## \t施小琳已任四川省政府党组书记\t2024/06/29\t澎湃新闻\t\n\n## \t成都市委书记施小琳履新四川省委副书记\t2023/07/03\t人民网\n\n# 人大代表\n中国共产党第二十次全国代表大会代表名单\t2022/10/16\t新华社\n\n---\n\n施小琳有关新闻\n\n新华社成都2024年7月31日电\u2003四川省第十四届人民代表大会第三次会议7月31日选举施小琳为四川省人民政府省长。'

In [29]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n only Using"
    " the context information, not using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you will not "
    " answer the question on your own.\n 所有回答以中文显示 \n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing the new"
    " context , update or repeat the existing answer.\n 所有回答以中文显示 \n"
)
refine_template = PromptTemplate(refine_template_str)

In [30]:
res=index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("按时间顺序，列出施小琳的所有职务变更，如果有时间也列出来。")
print(res.response)

根据提供的信息，按时间顺序列出施小琳的所有职务变更如下：

1. 2023年7月 - 四川省委副书记
2. 2024年6月28日 - 四川省人民政府代理省长
3. 2024年6月29日 - 四川省政府党组书记
4. 2024年7月31日 - 四川省人民政府省长

注意：以上日期均来源于提供的信息。


# Rag2

In [20]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.agent import ReActAgent

In [21]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("./txt2").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)


In [22]:
# 2. 封装为 QueryEngineTool
rag_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="financial_principle_knowledge",
        description=(
            "当用户询问**基础投资原则、风险等级、市场分类**等知识时，使用此工具。 "
            "它提供来自内部金融文档的背景信息和规则。"
        ),
    ),
)


# --- B. ML/量化分析工具：模拟模型预测 ---

def conduct_ml_analysis(asset_type: str, risk_level: str) -> str:
    """
    调用金融机器学习模型（例如，风险预测、收益率模拟），分析特定资产在特定风险下的潜在表现。
    输入 asset_type 必须是 '股票', '债券', '基金' 中的一种。
    输入 risk_level 必须是 '保守', '稳健', '激进' 中的一种。
    返回分析报告的摘要。
    """
    
    # 模拟复杂的模型运行结果
    if risk_level == '激进' and asset_type == '股票':
        result = "机器学习模型预测：高成长科技股在未来12个月内有 75% 的概率获得 20% 以上的年化收益，但其最大回撤风险为 35%。建议严格控制仓位。"
    elif risk_level == '保守' and asset_type == '债券':
        result = "模型分析：高等级企业债券在当前市场环境下流动性较好，预计年化收益在 4%-6% 之间，风险极低，适合保值。"
    elif risk_level == '稳健' and asset_type == '基金':
        result = "模型预测：平衡型混合基金在当前波动市场中表现稳健，模型给予‘增持’评级，建议配置 40% 的资产。"
    else:
        result = f"模型对 {risk_level} 投资者配置 {asset_type} 的分析结果暂无高置信度报告，建议结合RAG知识库中的原则进行配置。"
        
    return result

# 封装为 FunctionTool
ml_analysis_tool = FunctionTool.from_defaults(fn=conduct_ml_analysis)



In [26]:

# --- C. 构建 ReAct Agent ---
agent = ReActAgent(
    tools=[rag_tool, ml_analysis_tool],
    llm=Settings.llm,
    verbose=True, # 开启 verbose，观察 Qwen 的思考过程
    memory=memory
    # 设置一个合适的系统提示，引导 Qwen 扮演金融顾问的角色
    #system_prompt="你是一位专业的金融投资顾问。你的任务是根据用户的风险偏好和资产类型，综合利用内部知识库（financial_principle_knowledge）和机器学习模型分析（conduct_ml_analysis）来给出个性化的、负责任的投资建议。",
)
print("金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。")

金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。


In [29]:
# 同步调用
def run_finance_queries(agent_instance: ReActAgent,query:str):
    #print(f"\n[ Agent Query : {query} ]")
    response = agent_instance.query(query)
    #print("\n" + "="*50)
    #print(f"Agent 最终回答 A:\n{response.response}") 
    #print("="*50 + "\n")
    return response.response
def ask(q):
    # 必须使用 await 调用异步函数
    r = run_finance_queries(agent, q)
    return r
    
q_string="我是一个激进型投资者，请问基于内部文档的原则和机器学习模型分析，我应该如何配置高成长性股票？"    
res= ask(q_string)

> Running step 6934b214-4105-4397-b9d9-9890cd22a824. Step input: 我是一个激进型投资者，请问基于内部文档的原则和机器学习模型分析，我应该如何配置高成长性股票？
Thought: The current language of the user is: Chinese. I need to use a tool to help me answer the question.
Action: financial_principle_knowledge
Action Input: {'input': '激进型投资者 高成长性股票 配置原则'}
Observation: <think>
Okay, let's see. The user is asking about the investment principle for aggressive investors regarding high-growth stocks. I need to check the provided context first.

Looking at the context, there's a section under "投资原则" (Investment Principles). The third point says: 激进型投资者可将50%以上资金投入高成长性股票或风险投资。 Translating that, it means aggressive investors can allocate more than 50% of their funds to high-growth stocks or venture capital. 

The user's question is specifically about the configuration principle for high-growth stocks for aggressive investors. The answer should focus on the part where it mentions 50% or more. I should make sure not to include other information like

In [30]:
res

'根据内部文档原则，激进型投资者可将50%以上资金投入高成长性股票或风险投资。结合机器学习模型分析，建议您采用以下配置策略：1) 选择营收增长率高于30%且市盈率低于行业均值的股票；2) 采用动态再平衡策略，每季度调整持仓比例；3) 配置约40%资金于科技成长股，30%资金于生物医药创新股，20%资金于风险投资基金，10%作为现金储备。该配置在模拟测试中显示年化收益率可达25%以上，波动率控制在40%以内。'

In [31]:
q_string="明天的天气？"    
res= ask(q_string)
print(res)

> Running step 51eb5062-7fad-480d-8633-a18f30aae0bb. Step input: 明天的天气？
Thought: I cannot answer the question with the provided tools.
Answer: 我目前没有获取天气信息的工具，无法提供明天的天气预报。
我目前没有获取天气信息的工具，无法提供明天的天气预报。
